In [1]:
import pandas as pd
import csv
import re
from Bio import Entrez
from Bio import Medline
import numpy as np
import random
from sklearn import model_selection, preprocessing,linear_model,metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import RandomOverSampler, SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import MultinomialNB


df = pd.read_csv('litcovid2pubtator.xml.gz', compression='gzip', sep=' ', quotechar='"', error_bad_lines=False) # Covid-19 datasets


b'Skipping line 5: expected 89 fields, saw 146\nSkipping line 6: expected 89 fields, saw 3416\nSkipping line 7: expected 89 fields, saw 161\nSkipping line 8: expected 89 fields, saw 457\nSkipping line 9: expected 89 fields, saw 302\nSkipping line 10: expected 89 fields, saw 110\nSkipping line 12: expected 89 fields, saw 2546\nSkipping line 13: expected 89 fields, saw 2189\nSkipping line 14: expected 89 fields, saw 6869\nSkipping line 15: expected 89 fields, saw 217\nSkipping line 16: expected 89 fields, saw 183\nSkipping line 18: expected 89 fields, saw 447\nSkipping line 19: expected 89 fields, saw 664\nSkipping line 21: expected 89 fields, saw 155\nSkipping line 22: expected 89 fields, saw 636\nSkipping line 23: expected 89 fields, saw 2692\nSkipping line 24: expected 89 fields, saw 2253\nSkipping line 25: expected 89 fields, saw 1049\nSkipping line 27: expected 89 fields, saw 483\nSkipping line 28: expected 89 fields, saw 670\nSkipping line 30: expected 89 fields, saw 14703\nSkippin

In [1]:
conda install numpy

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install pandas

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
conda install -c conda-forge biopython

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\software

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.77             |   py38h1e8a9f7_1         2.6 MB  conda-forge
    conda-4.8.4                |   py38h32f6830_2         3.1 MB  conda-forge
    openssl-1.1.1g             |       he774522_0         5.7 MB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.5 MB

The following NEW packages will be INSTALLED:

  biopython          conda-forge/win-64::biopython-1.77-py38h1e8a9f7_1
  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.

In [4]:
conda install scikit-learn

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [6]:
conda install -c conda-forge lightgbm

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\software

  added / updated specs:
    - lightgbm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lightgbm-2.3.1             |   py38h6538335_0         555 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         555 KB

The following NEW packages will be INSTALLED:

  lightgbm           conda-forge/win-64::lightgbm-2.3.1-py38h6538335_0




lightgbm-2.3.1       | 555 KB    |            |   0% 
lightgbm-2.3.1       | 555 KB    | 2          |   3% 
lightgbm-2.3.1       | 555 KB    | ##3        |  23% 
lightgbm-2.3.1       | 555 KB    | #######4   |  75% 
lightgbm-2.3.1       | 555 KB    | ########## | 100% 
lightgbm-2.3.1       | 555 KB    | ########## | 100% 
Preparing transaction: ...working... done
Verifying transacti

In [7]:
conda install -c conda-forge imbalanced-learn

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\software

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    imbalanced-learn-0.7.0     |             py_1          97 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  imbalanced-learn   conda-forge/noarch::imbalanced-learn-0.7.0-py_1




imbalanced-learn-0.7 | 97 KB     |            |   0% 
imbalanced-learn-0.7 | 97 KB     | #6         |  17% 
imbalanced-learn-0.7 | 97 KB     | ########## | 100% 
imbalanced-learn-0.7 | 97 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the ke

In [ ]:
conda install -c anaconda py-xgboost

In [9]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
# I have 3 categories, label '1' is related to covid-19, 
df.index = df.index.map(str)
# get all content
word = ''
for i in range(1,5845):
    word += df.index[i]
# Use regular expression to extract pubmed id    
pattern = re.compile(r'3\d{6,7}')
pubmed_id = list(set(pattern.findall(word)))
# Get the content corresponding to pubmed id
Entrez.email = "qweng3@sheffield.ac.uk" # Always tell NCBI who you are 
idlist = pubmed_id
handle = Entrez.efetch(db="pubmed",id=idlist,rettype="medline",retmode="text")
records = Medline.parse(handle)

# Write the content in the pubmed database first and then read it
with open('output.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["article"])
    for record in records:
        writer.writerow([record])

df_text = pd.read_csv("output.csv")
# Separate contents into columns based on commas
df2_1 = df_text['article'].str.split(',', expand = True)

df2_1 = df2_1.where(df2_1.notnull(), np.nan).dropna(thresh = 4700, axis = 1)
include_index = df2_1[df2_1[3].notna()].index.tolist()
filter_df_1 = df_text.filter(items = include_index, axis = 0)
filter_df_1['label'] = 1
filter_df_1 

,article,label
0,"{'PMID': '32501898', 'OWN': 'NLM', 'STAT': 'Pu...",1
1,"{'PMID': '32587052', 'OWN': 'NLM', 'STAT': 'ME...",1
2,"{'PMID': '32227794', 'OWN': 'NLM', 'STAT': 'ME...",1
3,"{'PMID': '32478882', 'OWN': 'NLM', 'STAT': 'ME...",1
4,"{'PMID': '32250965', 'OWN': 'NLM', 'STAT': 'Pu...",1
...,...,...
5942,"{'PMID': '32420952', 'OWN': 'NLM', 'STAT': 'ME...",1
5943,"{'PMID': '32467100', 'OWN': 'NLM', 'STAT': 'ME...",1
5944,"{'PMID': '32410754', 'OWN': 'NLM', 'STAT': 'Pu...",1
5945,"{'PMID': '32579022', 'OWN': 'NLM', 'STAT': 'ME...",1


In [3]:

# label '0' is a randomly selected pubmedid, 
number_range = list(range(32000000,33000000))
pubmedid_0 = random.sample(number_range, 6877)
pubmedid_0 = [str(x) for x in pubmedid_0] # 下面代码要求必须得是str类型

Entrez.email = "qweng3@sheffield.ac.uk" # Always tell NCBI who you are 
idlist_0 = pubmedid_0
handle_0 = Entrez.efetch(db="pubmed",id=idlist_0,rettype="medline",retmode="text")
records_0 = Medline.parse(handle_0)


with open('output_0.csv', 'w') as f:
    writer_0 = csv.writer(f)
    writer_0.writerow(["article"])
    for record in records_0:
        writer_0.writerow([record])

df_text_0 = pd.read_csv("output_0.csv")
df2_0 = df_text_0['article'].str.split(',', expand = True)
df2_0 = df2_0.where(df2_0.notnull(), np.nan)
include_index_0 = df2_0[df2_0[3].notna()].index.tolist()

filter_df_0 = df_text_0.filter(items = include_index_0, axis = 0)
filter_df_0['label'] = 0
filter_df_0 

  

,article,label
0,"{'PMID': '32270326', 'OWN': 'NLM', 'STAT': 'In...",0
1,"{'PMID': '32518780', 'OWN': 'NLM', 'STAT': 'Pu...",0
2,"{'PMID': '32051494', 'OWN': 'NLM', 'STAT': 'Pu...",0
3,"{'PMID': '32002101', 'OWN': 'NLM', 'STAT': 'Pu...",0
5,"{'PMID': '32123107', 'OWN': 'NLM', 'STAT': 'ME...",0
...,...,...
6870,"{'PMID': '32253521', 'OWN': 'NLM', 'STAT': 'Pu...",0
6871,"{'PMID': '32776971', 'OWN': 'NLM', 'STAT': 'In...",0
6872,"{'PMID': '32306411', 'OWN': 'NLM', 'STAT': 'Pu...",0
6874,"{'PMID': '32317337', 'OWN': 'NLM', 'STAT': 'In...",0


In [4]:
# label '2' is the dataset in ScHARRHUD

df_SHR_raw = pd.read_csv('file.csv') # scharrhud datasets

df2_pubmedid = df_SHR_raw['Pubmed ID'].str.split(':',expand = True)[1].dropna().tolist()
Entrez.email = "qweng3@sheffield.ac.uk" # Always tell NCBI who you are 
idlist_SHR = df2_pubmedid
handle = Entrez.efetch(db="pubmed",id=idlist_SHR,rettype="medline",retmode="text")
records_SHR = Medline.parse(handle)

with open('output_SHR.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["article"])
    for record in records_SHR:
        writer.writerow([record])

df_SHR_text = pd.read_csv("output_SHR.csv")
df2_2 = df_SHR_text['article'].str.split(',', expand = True)
df2_2 = df2_2.where(df2_2.notnull(), np.nan)
df2_2 = df2_2.dropna(thresh = 1000, axis = 1)

include_index = df2_2[df2_2[3].notna()].index.tolist()
filter_df_2 = df_SHR_text.filter(items = include_index, axis = 0)
filter_df_2['label'] = 2
filter_df_2

,article,label
0,"{'PMID': '23549857', 'OWN': 'NLM', 'STAT': 'ME...",2
1,"{'PMID': '22886429', 'OWN': 'NLM', 'STAT': 'ME...",2
2,"{'PMID': '22054639', 'OWN': 'NLM', 'STAT': 'ME...",2
3,"{'PMID': '22381277', 'OWN': 'NLM', 'STAT': 'ME...",2
4,"{'PMID': '23231358', 'OWN': 'NLM', 'STAT': 'ME...",2
...,...,...
1043,"{'PMID': '25395984', 'OWN': 'NLM', 'STAT': 'Pu...",2
1044,"{'PMID': '25395940', 'OWN': 'NLM', 'STAT': 'Pu...",2
1045,"{'PMID': '25368853', 'OWN': 'NLM', 'STAT': 'Pu...",2
1046,"{'PMID': '28955695', 'OWN': 'NLM', 'STAT': 'Pu...",2


In [5]:
df_concat = pd.concat([filter_df_1, filter_df_0, filter_df_2], axis=0 )


In [ ]:
[df_concat['article']]

In [6]:
train_x,valid_x, train_y, valid_y = model_selection.train_test_split(df_concat['article'], df_concat['label'])
# tfidf method to get feature 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df_concat['article'])
xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tf_idf = tfidf_vect.transform(valid_x)

In [ ]:
# # Over-Sampling method due to unbalanced data
# smote = SMOTE(random_state=42)
# xtrain_tfidf_res, train_y_res = smote.fit_resample(xtrain_tfidf, train_y) 
# xtrain_tfidf = xtrain_tfidf_res
# train_y = train_y_res

In [ ]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV

# parameters = {"n_estimators": np.arange(200,500,50)
#              , "learning_rate": np.arange(0.008, 0.02, 0.002)
# #            , "max_depth": np.arange(6,9,1)
# #             , "gamma": np.arange(0.5,2,0.3)
# #              , "reg_lambda": np.arange(0.5,2, 0.3)
# #               ,"reg_alpha": np.arange(0.5, 2, 0.2)
#              }

# reg =  XGBClassifier(n_estimators = 450
#                     , early_stopping_rounds = 5
#                     , learning_rate = 0.014 
#                     , max_depth = 6
#                     , objective='multi:softmax'   #'binary:logistic'
#                     , reg_lambda = 0.5
#                     , gamma = 0.5
#                     )
# GS = GridSearchCV(reg, parameters, cv=3,scoring = 'recall_weighted')
# GS.fit(xtrain_tfidf, train_y)
# GS.best_params_

In [ ]:

# parameters_lgbm = {"n_estimators": np.arange(100,500,50)
#              , "learning_rate": np.arange(0.006, 0.02, 0.002)
# #             , "max_depth": np.arange(6,20,2)
# #              , "reg_lambda": np.arange(0.5,2, 0.3)
# #              , "subsample": np.arange(0,1,0.2)
# #              , "reg_alpha": np.arange(0.5, 2, 0.2)
# #              , "num_leaves": np.arange(10,300, 50) 
#              }

# reg_lgbm =  LGBMClassifier(learning_rate = 0.006
#                            ,objective='multiclass' #'binary'
#                            ,n_estimators = 100
#                            ,max_depth = 6
#                            ,reg_alpha = 0.5
#                            ,reg_lambda = 0,5
#                            ,objective= 'binary'))
# GS_lgbm = GridSearchCV(reg_lgbm, parameters_lgbm, cv=3, scoring = 'recall_weighted')
# GS_lgbm.fit(xtrain_tfidf, train_y)
# GS_lgbm.best_params_

In [ ]:
# If use binary classificationi, we can run this part code, it's logisticRegression
# LogisticRegression
# logreg = linear_model.LogisticRegression()
# logreg.fit(xtrain_tfidf, train_y)
# predictions = logreg.predict(xvalid_tf_idf)
# accuracy = metrics.accuracy_score(predictions, valid_y)
# print("LR, WordLevel TF-IDF\nAccuracy: ", accuracy)
# recall_score = metrics.recall_score(predictions,valid_y)
# print("LR, WordLevel TF-IDF\nrecallscore: ",recall_score)
# cf_matrix = metrics.confusion_matrix(predictions, valid_y)
# print(cf_matrix)
# print(metrics.classification_report(valid_y, predictions, target_names=['irrel', 'rel']))

In [11]:
# XGBoost classification
from xgboost import XGBClassifier


XGB_model = XGBClassifier( n_estimators = 200
                         , early_stopping_rounds = 5
                         , learning_rate = 0.014
                         , max_depth = 5
                         , objective= 'multi:softmax' 
                          #'binary:logistic'
                         , reg_alpha = 0.7
                         , reg_lambda = 1.4
                         , gamma = 0.5
                          )
XGB_model.fit(xtrain_tfidf, train_y)
valid_preds = XGB_model.predict(xvalid_tf_idf)
predictions = [round(value) for value in valid_preds]
test_accuracy_xgboost = metrics.accuracy_score(predictions, valid_y)
print("XGBoost, WordLevel TF-IDF\nAccuracy: ", test_accuracy_xgboost)
recall_score_xgboost = metrics.recall_score(predictions,valid_y, average = 'weighted')
print("XGBoost, WordLevel TF-IDF\nrecallscore: ",recall_score_xgboost)

[07:02:30] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { early_stopping_rounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


XGBoost, WordLevel TF-IDF
Accuracy:  0.9744081893793985
XGBoost, WordLevel TF-IDF
recallscore:  0.9744081893793985


In [7]:


lgbm_model = LGBMClassifier(learning_rate = 0.007
                           ,objective = 'multiclass' 
                            # 'binary'
                           ,n_estimators = 250
                           ,max_depth = 8
                           ,reg_alpha = 0.8
                           ,reg_lambda = 1.8)
lgbm_model.fit(xtrain_tfidf, train_y)
valid_preds_lgbm = lgbm_model.predict(xvalid_tf_idf)
predictions_lgbm = [round(value) for value in valid_preds_lgbm]
test_accuracy_lgbm = metrics.accuracy_score(predictions_lgbm, valid_y)
print("LGBM, WordLevel TF-IDF\nAccuracy: ", test_accuracy_lgbm)
recall_score_lgbm = metrics.recall_score(predictions_lgbm,valid_y,average = 'weighted')
print("LGBM, WordLevel TF-IDF\nrecallscore: ",recall_score_lgbm)


LGBM, WordLevel TF-IDF
Accuracy:  0.9737683941138836
LGBM, WordLevel TF-IDF
recallscore:  0.9737683941138836


In [8]:

clf=MultinomialNB(alpha=1.0,fit_prior=True)
clf.fit(xtrain_tfidf, train_y)
valid_clf_preds = clf.predict(xvalid_tf_idf)
predictions_clf = [round(value) for value in valid_clf_preds]
accuracy_clf = metrics.accuracy_score(predictions_clf,valid_y)
recall_score_clf = metrics.recall_score(predictions_clf, valid_y, average = 'weighted')
print("NB, WordLevel TF-IDF\nAccuracy: ", accuracy_clf)
print("NB, WordLevel TF-IDF\nrecallscore: ",recall_score_clf)


NB, WordLevel TF-IDF
Accuracy:  0.9193857965451055
NB, WordLevel TF-IDF
recallscore:  0.9193857965451055
